In [ ]:
# ##############################################################################
#
# 통합 펩타이드 발굴 파이프라인 (Single-Notebook Peptide Discovery Pipeline) - PRODIGY 버전
#
# 이 코드는 Google Colab Pro/Pro+ 환경에서 실행하는 것을 권장합니다.
# Colab 메뉴에서 '런타임' -> '런타임 유형 변경'을 선택하여
# 하드웨어 가속기를 'GPU'로, 런타임 구성을 '높은 RAM'으로 설정해주세요.
#
# ##############################################################################

# ==============================================================================
# CELL 1: 환경 설정, 설치 및 라이브러리 임포트 (최초 1회만 실행)
# ==============================================================================

# --- [1/5] 모든 라이브러리 임포트 ---
import os
import sys
import site
import subprocess
import shutil
import time
import glob
import json
import re
import torch
from datetime import datetime
import pytz
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForMaskedLM
import pandas as pd
from IPython.display import display
import numpy as np
import requests
from bs4 import BeautifulSoup


# --- [2/5] 환경 설정 및 라이브러리 설치 ---
print("="*80)
print("STEP 0: 환경 설정 및 모든 필수 라이브러리 설치")
print("="*80)

print("\n   > ColabFold 및 필수 라이브러리를 설치합니다...")
# ColabFold (TensorFlow 충돌 방지 포함)
os.system("pip uninstall -y tensorflow tensorboard tb-nightly tensorflow-estimator tensorflow-hub tensorflow-io > /dev/null 2>&1")
os.system("pip install -q --no-warn-conflicts 'colabfold[alphafold] @ git+https://github.com/sokrypton/ColabFold'")
os.system("pip install -q --no-warn-conflicts 'jax[cuda11_pip]' -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html")

# ESM-2
os.system("pip install -q --upgrade transformers sentencepiece")

# 결합력 평가 도구 (OpenBabel, RDKit, PLIP 등)
os.system("apt-get update -qq > /dev/null 2>&1 && apt-get install -y --quiet openbabel python3-openbabel libopenbabel-dev > /dev/null 2>&1")
os.system("pip install -q openbabel-wheel rdkit-pypi plip")

# 의존성 해결을 위한 라이브러리
os.system("pip install -q oddt scikit-learn")

# Vina 관련 및 기타 도구
os.system("pip install -q pymol-open-source meeko > /dev/null 2>&1")
os.system("pip install -q pytz openpyxl requests beautifulsoup4")
print("   > 라이브러리 설치 완료.")


# --- [3/5] ColabFold 패치 적용 ---
print("\n   > ColabFold 스크립트 패치 적용 중...")
try:
    dist_packages_path = site.getsitepackages()[0]
    batch_py_path = os.path.join(dist_packages_path, 'colabfold', 'batch.py')
    if os.path.exists(batch_py_path):
        os.system(f"sed -i 's/tf.get_logger().setLevel(logging.ERROR)/#tf.get_logger().setLevel(logging.ERROR)/g' {batch_py_path}")
        os.system(f"sed -i \\\"s/tf.config.set_visible_devices(\\\\\\\\[\\\\\\\\], 'GPU')/#tf.config.set_visible_devices(\\\\\\\\[\\\\\\\\], 'GPU')/g\\\" {batch_py_path}")
        print("   > 패치 적용 완료.")
    else:
        print(f"   > 경고: {batch_py_path}를 찾을 수 없어 패치를 건너뜁니다.")
except Exception as e:
    print(f"   > 경고: ColabFold 패치 중 오류 발생 - {e}")

# --- [4/5] AutoDock Vina 설치 ---
print("\n   > AutoDock Vina 설치 중...")
def setup_vina_robust():
    vina_dir = "vina_1.2.3_linux_x86_64"
    if not os.path.exists(vina_dir):
        print("   > Vina 다운로드 중...")
        os.system("wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.3/vina_1.2.3_linux_x86_64.zip -O vina.zip")
        os.system("unzip -q -o vina.zip")
    vina_executable = os.path.abspath(f"./{vina_dir}/bin/vina")
    if os.path.exists(vina_executable):
        os.chmod(vina_executable, 0o755)
        print(f"   > Vina 실행파일 발견: {vina_executable}")
        return vina_executable
    else:
        print("   > Vina 실행파일을 찾을 수 없음")
        return None
VINA_EXECUTABLE = setup_vina_robust()

# --- [5/5] PLIP 확인 및 모델 로딩 ---
print("\n   > PLIP 확인 및 모델 로딩 중...")
try:
    from plip.structure.preparation import PDBComplex
    USE_PLIP = True
    print("   > PLIP 임포트 성공!")
except ImportError:
    USE_PLIP = False
    print("   > PLIP 임포트 실패.")

model_name = "facebook/esm2_t12_35M_UR50D"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
print("   > ESM-2 모델 로딩 완료.")

print("\n" + "="*80)
print("✅ 모든 준비가 완료되었습니다. 이제 아래의 '메인 파이프라인' 셀을 실행해주세요.")
print("="*80)

STEP 0: 환경 설정 및 모든 필수 라이브러리 설치

   > ColabFold 및 필수 라이브러리를 설치합니다...
   > 라이브러리 설치 완료.

   > ColabFold 스크립트 패치 적용 중...
   > 패치 적용 완료.

   > AutoDock Vina 설치 중...
   > Vina 다운로드 중...
   > Vina 실행파일을 찾을 수 없음

   > PLIP 확인 및 모델 로딩 중...
   > PLIP 임포트 실패.
   > ESM-2 모델 로딩 완료.

✅ 모든 준비가 완료되었습니다. 이제 아래의 '메인 파이프라인' 셀을 실행해주세요.


In [ ]:
# ==============================================================================
# CELL 2: 메인 파이프라인 (설정 변경 후 반복 실행 가능)
# ==============================================================================

# 파이프라인 시작 시간 기록
pipeline_start_time = time.time()

# ==============================================================================
# STEP 1: 파이프라인 실행을 위한 변수 설정
# ==============================================================================
print("\n" + "="*80)
print("STEP 1: 파이프라인 실행을 위한 변수 설정")
print("="*80)

# --- ⚙️ 사용자 설정 영역 ---
N_PEPTIDES = 10
TARGET_PROTEIN_SEQUENCE = "AFTVTVPKDLYVVEYGSNMTIECKFPVEKQLDLAALIVYWEMEDKNIIQFVHGEEDLKVQHSSYRQRARLLKDQLSLGNAALQITDVKLQDAGVYRCMISYGGADYKRITVKVNAPYNKINQRILVVDPVTSEHELTCQAEGYPKAEVIWTSSDHQVLSGKTTTTNSKREEKLFNVTSTLRINTTTNEIFYCTFRRLDPEENHTAELVIPELPLAHPPNERT"
PEPTIDE_LENGTH = 4
BASE_FOLDER_PREFIX = "PDP"
# --- 설정 종료 ---

# --- 경로 설정 (요청사항 반영) ---
kst = pytz.timezone('Asia/Seoul')
timestamp = datetime.now(kst).strftime("%Y%m%d_%H%M%S")
JOB_NAME = f"{BASE_FOLDER_PREFIX}_{timestamp}"

# 하위 폴더 생성
fasta_folder = os.path.join(JOB_NAME, "PROTEIN_FASTA")
os.makedirs(fasta_folder, exist_ok=True)
final_folder = os.path.join(JOB_NAME, "final")
os.makedirs(final_folder, exist_ok=True)

# 파일 경로 변수 수정
PROTEIN_FASTA_PATH = os.path.join(fasta_folder, "target_protein.fasta")
OUTPUT_FINAL_XLSX_PATH = os.path.join(final_folder, f"final_peptide_ranking_{timestamp}.xlsx")

with open(PROTEIN_FASTA_PATH, "w") as f:
    f.write(f">target_protein\n{TARGET_PROTEIN_SEQUENCE}\n")

print(f"✔️ 작업 폴더: {JOB_NAME}")
print(f"✔️ FASTA 저장 폴더: {fasta_folder}")
print(f"✔️ 최종 결과 저장 폴더: {final_folder}")
print("="*80)
print("✅ STEP 1: 설정이 완료되었습니다.")
print("="*80)


# ==============================================================================
# STEP 2: PepMLM (ESM-2)을 이용한 타겟 특이적 펩타이드 후보 생성 (요청 코드로 수정)
# ==============================================================================
print("\n" + "="*80)
print("STEP 2: PepMLM (ESM-2)을 이용한 타겟 특이적 펩타이드 후보 생성")
print("="*80)

# 생성 파라미터
temperature = 1.0
top_k = 50

# 모델 입력용 프롬프트 생성
formatted_target = " ".join(list(TARGET_PROTEIN_SEQUENCE))
mask_tokens = " ".join([tokenizer.mask_token] * PEPTIDE_LENGTH)
prompt = f"{tokenizer.cls_token} {formatted_target} {tokenizer.eos_token} {mask_tokens}"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

mask_token_indices = (input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

peptides = []
peptide_fasta_paths = []

print("\n펩타이드 서열 생성을 시작합니다...")
with torch.no_grad():
    for i in range(N_PEPTIDES):
        current_ids = input_ids.clone().to(model.device)
        shuffled_mask_indices = mask_token_indices[torch.randperm(len(mask_token_indices))]

        for mask_idx in shuffled_mask_indices:
            outputs = model(input_ids=current_ids)
            logits = outputs.logits
            mask_logits = logits[0, mask_idx, :]
            filtered_logits = mask_logits / temperature
            effective_top_k = min(top_k, tokenizer.vocab_size)
            top_k_values, top_k_indices = torch.topk(filtered_logits, effective_top_k)
            filter_tensor = torch.full_like(filtered_logits, -float('Inf'))
            filter_tensor.scatter_(0, top_k_indices, top_k_values)
            probs = F.softmax(filter_tensor, dim=-1)
            predicted_token_id = torch.multinomial(probs, num_samples=1)
            current_ids[0, mask_idx] = predicted_token_id.item()

        generated_token_ids = current_ids[0, mask_token_indices]
        sequence_part = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
        sequence = "".join(sequence_part.split())
        peptides.append(sequence)

        # 생성된 펩타이드를 개별 fasta 파일로 저장
        peptide_fasta_folder = os.path.join(JOB_NAME, "peptide_fastas")
        os.makedirs(peptide_fasta_folder, exist_ok=True)
        fasta_path = os.path.join(peptide_fasta_folder, f"peptide_{i}.fasta")
        with open(fasta_path, "w") as f:
            f.write(f">peptide_{i}\n{sequence}\n")
        peptide_fasta_paths.append(fasta_path)
        print(f"  [{i+1}/{N_PEPTIDES}] 생성 완료: {sequence} (길이: {len(sequence)})")

print("\n--- 생성된 펩타이드 후보 목록 ---")
for i, seq in enumerate(peptides):
    print(f"  - 후보 {i+1}: {seq}")
print("="*80)
print(f"✅ STEP 2: 총 {N_PEPTIDES}개의 펩타이드 후보 생성을 완료했습니다.")
print("="*80)


# ==============================================================================
# STEP 3: 단백질-펩타이드 복합체 3D 구조 예측 (ColabFold)
# ==============================================================================
print("\n" + "="*80)
print("STEP 3: 단백질-펩타이드 복합체 3D 구조 예측 (ColabFold)")
print("="*80)

batch_csv_path = os.path.join(JOB_NAME, "batch_complexes.csv")
with open(batch_csv_path, "w") as f:
    f.write("id,sequence\n")
    for i, peptide_seq in enumerate(peptides):
        f.write(f"complex_{i},{TARGET_PROTEIN_SEQUENCE}:{peptide_seq}\n")

output_dir = os.path.join(JOB_NAME, "colabfold_batch_output")
log_file = os.path.join(output_dir, "colabfold_batch.log")
colabfold_cmd = (f"colabfold_batch --num-recycle 1 --model-type alphafold2_multimer_v3 "
                f"--rank ptm --max-msa 32:128 --num-models 1 --stop-at-score 0.5 "
                f"{batch_csv_path} {output_dir} > {log_file} 2>&1")

print(f"\nColabFold 배치 실행 시작... (예상 소요 시간: 10-30분)")
result = os.system(colabfold_cmd)

if result != 0:
    print(f"\n🚨 ColabFold 실행 실패! (종료 코드: {result})")
    if os.path.exists(log_file):
        print("="*40, "\n오류 로그:\n", "="*40)
        with open(log_file, 'r') as f: print(f.read())
        print("="*40)
else:
    print(f"\nColabFold 실행 완료 (종료 코드: {result})")

predicted_pdb_files = sorted(glob.glob(os.path.join(output_dir, "complex_*_unrelaxed_rank_001*.pdb")))
print(f"총 {len(predicted_pdb_files)}개의 3D 구조 예측을 확인했습니다.")
print("="*80)
print(f"✅ STEP 3: 3D 구조 예측이 완료되었습니다.")
print("="*80)


# ==============================================================================
# STEP 3.5: 구조 예측 신뢰도 점수(pTM) 확인
# ==============================================================================
print("\n" + "="*80)
print("STEP 3.5: 구조 예측 신뢰도 점수(pTM) 확인")
print("="*80)
ptm_scores_map = {}
all_score_files = sorted(glob.glob(os.path.join(output_dir, "*_scores.json")))
print(f"총 {len(all_score_files)}개의 점수 파일을 분석합니다...")
for score_file in all_score_files:
    try:
        match = re.search(r'complex_(\d+)', os.path.basename(score_file))
        if match:
            peptide_index = int(match.group(1))
            with open(score_file, 'r') as f: data = json.load(f)
            ptm_score = data.get('ptm', data.get('iptm', [0.0]))[0]
            if peptide_index < len(peptides):
                ptm_scores_map[peptides[peptide_index]] = round(float(ptm_score), 3)
    except Exception as e:
        print(f"오류: {os.path.basename(score_file)} 처리 중 문제 발생 - {e}")
print("="*80)
print("✅ STEP 3.5: pTM 점수 확인이 완료되었습니다.")
print("="*80)


# ==============================================================================
# STEP 4: 결합력 평가 및 최종 랭킹 계산
# ==============================================================================
print("\n" + "="*80)
print("STEP 4: 결합력 평가 및 최종 랭킹 계산")
print("="*80)
# --- 평가 함수들 (첨부파일과 동일) ---
def calculate_interactions_with_plip(pdb_file):
    try:
        mol = PDBComplex(); mol.load_pdb(pdb_file); mol.analyze()
        interactions = mol.interaction_sets
        h_bonds, hydrophobic, electrostatic = 0, 0, 0
        for inter in interactions.values():
            if inter.ligchain == 'B':
                h_bonds += len(inter.hbonds_pdon) + len(inter.hbonds_ldon)
                hydrophobic += len(inter.hydrophobic_contacts)
                electrostatic += len(inter.saltbridge_lneg) + len(inter.saltbridge_pneg)
        total = h_bonds + hydrophobic + electrostatic
        return {'h_bonds': h_bonds, 'hydrophobic': hydrophobic, 'electrostatic': electrostatic, 'total': total}
    except Exception as e:
        print(f"       PLIP 분석 오류: {e}. 대체 함수로 전환합니다.")
        return None
def calculate_interactions_advanced(pdb_file):
    try:
        chain_a_atoms, chain_b_atoms = [], []
        with open(pdb_file, 'r') as f:
            for line in f:
                if line.startswith(('ATOM', 'HETATM')):
                    chain, element = line[21], (line[76:78].strip() or line[12:16].strip()[0]).upper()
                    coords = (float(line[30:38]), float(line[38:46]), float(line[46:54]), element)
                    if chain == 'A': chain_a_atoms.append(coords)
                    elif chain == 'B': chain_b_atoms.append(coords)
        h_bonds, hydrophobic = 0, 0
        for ax, ay, az, a_elem in chain_a_atoms:
            for bx, by, bz, b_elem in chain_b_atoms:
                dist = np.sqrt((ax-bx)**2 + (ay-by)**2 + (az-bz)**2)
                if dist <= 3.5 and a_elem in 'NO' and b_elem in 'NO': h_bonds += 1
                if dist <= 4.5 and a_elem == 'C' and b_elem == 'C': hydrophobic += 1
        total = h_bonds + hydrophobic
        return {'h_bonds': h_bonds, 'hydrophobic': hydrophobic, 'electrostatic': 0, 'total': total}
    except Exception: return {'h_bonds': 0, 'hydrophobic': 0, 'electrostatic': 0, 'total': 0}
def predict_binding_affinity_with_prodigy(pdb_file_path, max_retries=3):
    url = "https://bianca.science.uu.nl/prodigy/"
    for attempt in range(max_retries):
        try:
            with open(pdb_file_path, 'rb') as f:
                files = {'file_upload': (os.path.basename(pdb_file_path), f)}
                data = {'chain1': 'A', 'chain2': 'B', 'contact_builder': 'contact_and_predict'}
                response = requests.post(url, files=files, data=data, timeout=60)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                affinity_header = soup.find('th', string=re.compile(r'Predicted binding affinity'))
                if affinity_header: return float(affinity_header.find_next_sibling('td').text.strip())
                return 0.0
            else: print(f"       PRODIGY 서버 에러: 상태 코드 {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"       PRODIGY 요청 에러 (시도 {attempt + 1}): {e}")
            time.sleep(5)
    return 0.0
def run_vina_docking(pdb_file, base_name, vina_executable):
    try:
        receptor_pdb, ligand_pdb = f"{base_name}_receptor.pdb", f"{base_name}_ligand.pdb"
        with open(pdb_file, 'r') as f_in, open(receptor_pdb, 'w') as f_r, open(ligand_pdb, 'w') as f_l:
            chain_b_coords = []
            for line in f_in:
                if line.startswith(('ATOM', 'HETATM')):
                    if line[21] == 'A': f_r.write(line)
                    elif line[21] == 'B':
                        f_l.write(line)
                        chain_b_coords.append([float(line[30:38]), float(line[38:46]), float(line[46:54])])
        if not chain_b_coords: return 0.0
        center = np.mean(chain_b_coords, axis=0).tolist()
        receptor_pdbqt, ligand_pdbqt = f"{base_name}_receptor.pdbqt", f"{base_name}_ligand.pdbqt"
        os.system(f"mk_prepare_receptor -i {receptor_pdb} -o {receptor_pdbqt} > /dev/null 2>&1")
        os.system(f"mk_prepare_ligand -i {ligand_pdb} -o {ligand_pdbqt} --rigid > /dev/null 2>&1")
        if not (os.path.exists(receptor_pdbqt) and os.path.exists(ligand_pdbqt)): return 0.0
        log_file = f"{base_name}_vina.log"
        cmd = f"{vina_executable} --receptor {receptor_pdbqt} --ligand {ligand_pdbqt} --center_x {center[0]} --center_y {center[1]} --center_z {center[2]} --size_x 30 --size_y 30 --size_z 30 --exhaustiveness 16 --log {log_file} >/dev/null 2>&1"
        os.system(cmd)
        with open(log_file, 'r') as f:
            for line in f:
                if line.strip().startswith('1'): return float(line.split()[1])
    except Exception as e: print(f"       Vina 도킹 중 오류: {e}")
    return 0.0
results = []
print(f"총 {len(predicted_pdb_files)}개의 구조에 대해 평가를 시작합니다...")
for idx, pdb_file in enumerate(predicted_pdb_files):
    print(f"\n  평가 중 ({idx+1}/{len(predicted_pdb_files)}): {os.path.basename(pdb_file)}")
    base_name, peptide_seq = os.path.join(JOB_NAME, f"eval_{idx}"), peptides[idx]
    prodigy_score = predict_binding_affinity_with_prodigy(pdb_file)
    print(f"    -> PRODIGY 점수: {prodigy_score:.3f} kcal/mol")
    vina_score = run_vina_docking(pdb_file, base_name, VINA_EXECUTABLE) if VINA_EXECUTABLE else 0.0
    print(f"    -> Vina 점수: {vina_score:.3f} kcal/mol")
    interactions = calculate_interactions_with_plip(pdb_file) if USE_PLIP else None
    if interactions is None: interactions = calculate_interactions_advanced(pdb_file)
    print(f"    -> 상호작용: H-bonds={interactions['h_bonds']}, Hydrophobic={interactions['hydrophobic']}, Electrostatic={interactions['electrostatic']}")
    final_score = (abs(prodigy_score) * 0.5 + abs(vina_score) * 0.25 + interactions['total'] * 0.15 + ptm_scores_map.get(peptide_seq, 0.0) * 10 * 0.1)
    results.append({"Peptide Sequence": peptide_seq, "Final Score": round(final_score, 3), "PRODIGY Score (kcal/mol)": round(prodigy_score, 3), "Vina Score (kcal/mol)": round(vina_score, 3), "Total Interactions": interactions['total'], "pTM Score": ptm_scores_map.get(peptide_seq, 0.0), "H-bonds": interactions['h_bonds'], "Hydrophobic": interactions['hydrophobic'], "Electrostatic": interactions['electrostatic'], "Source PDB": os.path.basename(pdb_file)})
    print(f"    -> 평가 완료: Final Score = {final_score:.3f}")
print("="*80)
print("✅ STEP 4: 모든 구조에 대한 평가 및 점수 계산을 완료했습니다.")
print("="*80)


# ==============================================================================
# STEP 5: 최종 결과 확인 및 저장
# ==============================================================================
print("\n" + "="*80)
print("STEP 5: 최종 결과 확인 및 저장")
print("="*80)
if results:
    df = pd.DataFrame(results)
    column_order = ["Peptide Sequence", "Final Score", "PRODIGY Score (kcal/mol)", "Vina Score (kcal/mol)", "Total Interactions", "pTM Score", "H-bonds", "Hydrophobic", "Electrostatic", "Source PDB"]
    df_final = df.sort_values("Final Score", ascending=False).reset_index(drop=True)
    df_final = df_final[[col for col in column_order if col in df_final.columns]]
    df_final.to_excel(OUTPUT_FINAL_XLSX_PATH, index=False)
    print("\n🏆 최종 펩타이드 후보 랭킹:")
    display(df_final)
    print(f"\n💾 전체 결과가 Excel 파일로 저장되었습니다: {OUTPUT_FINAL_XLSX_PATH}")
    print("\n📊 결과 요약:")
    if not df_final.empty:
        print(f"   • 총 평가된 펩타이드: {len(results)}개")
        print(f"   • 최고 점수 펩타이드: {df_final.iloc[0]['Peptide Sequence']} (점수: {df_final.iloc[0]['Final Score']:.3f})")
        print(f"   • 평균 PRODIGY 점수: {df_final['PRODIGY Score (kcal/mol)'].mean():.3f} kcal/mol")
        print(f"   • 평균 상호작용 수: {df_final['Total Interactions'].mean():.1f}")
else: print("\n❌ 최종 결과가 없습니다.")
print("="*80)
print("🎉 모든 파이프라인 실행이 완료되었습니다!")
print("="*80)
print("\n📋 설치된 도구 상태:")
print(f"   • ColabFold: ✅ 설치됨"); print(f"   • ESM-2: ✅ 설치됨"); print(f"   • PRODIGY: ✅ 웹 API 통합")
print(f"   • AutoDock Vina: {'✅ 설치됨' if VINA_EXECUTABLE else '⚠️ 사용 안 함'}")
print(f"   • PLIP: {'✅ 설치됨 (우선 사용)' if USE_PLIP else '⚠️ 설치 실패 (대체 함수 사용)'}")
print("="*80)
pipeline_end_time = time.time()
total_duration_seconds = pipeline_end_time - pipeline_start_time
total_minutes = int(total_duration_seconds // 60)
total_seconds = int(total_duration_seconds % 60)
print(f"\n⏱️  총 실행 시간: {total_minutes}분 {total_seconds}초")
print("="*80)


STEP 1: 파이프라인 실행을 위한 변수 설정
✔️ 작업 폴더: PDP_20250915_180245
✔️ FASTA 저장 폴더: PDP_20250915_180245/PROTEIN_FASTA
✔️ 최종 결과 저장 폴더: PDP_20250915_180245/final
✅ STEP 1: 설정이 완료되었습니다.

STEP 2: PepMLM (ESM-2)을 이용한 타겟 특이적 펩타이드 후보 생성

펩타이드 서열 생성을 시작합니다...
  [1/10] 생성 완료: KYSM (길이: 4)
  [2/10] 생성 완료: QINR (길이: 4)
  [3/10] 생성 완료: KRHS (길이: 4)
  [4/10] 생성 완료: EEEW (길이: 4)
  [5/10] 생성 완료: KQLL (길이: 4)
  [6/10] 생성 완료: FQKC (길이: 4)
  [7/10] 생성 완료: HTQC (길이: 4)
  [8/10] 생성 완료: KPPL (길이: 4)
  [9/10] 생성 완료: SARG (길이: 4)
  [10/10] 생성 완료: LFYI (길이: 4)

--- 생성된 펩타이드 후보 목록 ---
  - 후보 1: KYSM
  - 후보 2: QINR
  - 후보 3: KRHS
  - 후보 4: EEEW
  - 후보 5: KQLL
  - 후보 6: FQKC
  - 후보 7: HTQC
  - 후보 8: KPPL
  - 후보 9: SARG
  - 후보 10: LFYI
✅ STEP 2: 총 10개의 펩타이드 후보 생성을 완료했습니다.

STEP 3: 단백질-펩타이드 복합체 3D 구조 예측 (ColabFold)

ColabFold 배치 실행 시작... (예상 소요 시간: 10-30분)

🚨 ColabFold 실행 실패! (종료 코드: 512)
총 0개의 3D 구조 예측을 확인했습니다.
✅ STEP 3: 3D 구조 예측이 완료되었습니다.

STEP 3.5: 구조 예측 신뢰도 점수(pTM) 확인
총 0개의 점수 파일을 분석합니다...
✅ STEP 3.5: pTM 점수 확인이 완료되었습니다